# Practical session on Adversarial Machine Learning
**Summer School Cyber in Font-Romeu**

July 9th 2025

### Imports and parameters

In [300]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
# For some reason, we need to run this
tf.config.run_functions_eagerly(True)
tf.compat.v1.enable_eager_execution()

from keras.models import load_model
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from keras.optimizers import Adam

from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, BoundaryAttack
from art.defences.detector.evasion import BinaryInputDetector
from art.defences.trainer import AdversarialTrainer

import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from utils import random_subsample, show_images, plot_images_pca, create_model_cifar, load_cifar100_file, plot_adversarial_comparison

In [301]:
TARGET_CLASS=b'people'

In [303]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Adversarial machine learning: the science of fooling machine learning models.

The two main aspects of adversarial machine learning are:
1. **Attacks:** Techniques to generate inputs that cause a model to make wrong predictions with high confidence. These can be:
    - Evasion attacks (modifying inputs at test time, e.g., adding small perturbations to images to fool a classifier),
    - Poisoning attacks (manipulating the training data to embed vulnerabilities),
    - Exploratory attacks (extracting information about the model, e.g., through model inversion or membership inference).
2. **Defenses:** Strategies to make models more robust or to detect threats. Examples include
    - adversarial training (training on perturbed data),
    - detection of poisoning and adversarial examples,
    - certified robustness approaches.

Note that poisoning, membership inference and the like where explained by Josep Domingo Ferrer on monday.

<img src="https://raw.githubusercontent.com/Trusted-AI/adversarial-robustness-toolbox/refs/heads/main/docs/images/white_hat_blue_red.png" alt="Red and Blue teams for Adversarial ML" width = 400/>

In this session, we will use the Adversarial Robustness Toolbox (ART) developed by IBM: https://github.com/Trusted-AI/adversarial-robustness-toolbox
- It supports multiple attack and defense algorithms,
- It is actively maintained and well-documented,
- It works with major deep learning frameworks (TensorFlow, PyTorch, Keras).

*Comparative table of the main adversarial ML libraries*
| Library | Supported Frameworks | Attacks & Defenses | Ease of Use | Actively | Maintained |
| --- | --- | --- | --- | --- | --- |
| Adversarial Robustness Toolbox (ART) | TensorFlow, PyTorch, Keras, MXNet | Many (FGSM, PGD, CW, etc.) | Many | High | ✅|
| CleverHans | TensorFlow, PyTorch | Evasion (many), less focus on defenses | Few | Moderate | ⚠️ (less active)|
| Foolbox | TensorFlow, PyTorch | Wide variety (esp. evasion) | Limited | High | ✅|
| Advertorch | PyTorch | Evasion-focused | Some defenses | High | ⚠️ (slightly less active)|
| DeepRobust | PyTorch | Evasion, poisoning | Several | Moderate | ✅ |

# Part 1: The classification of images with a class "people"

## 1.1 Dataset: CIFAR100

We use here the CIFAR-100 dataset, a well-known benchmark dataset for image classification tasks.

*What is in CIFAR-100?*
- It contains 60,000 color images of size 32×32 pixels.
- Each image belongs to one of 100 classes (for example, apple, castle, lion).
- These 100 fine classes are grouped into 20 super classes, which represent more general categories (for example, food containers, large carnivores, people).

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20240522153720/download.png" alt="CIFAR100" width = 250/>

The dataset is split as follows:
- 50,000 training images
- 10,000 test images

*Focus on the “people” class*

Among the 20 super classes, there is a group called “people”, which gathers all images containing humans.

In this session, we will focus on attacking this "people" class.
Our goal is to study how adversarial examples can mislead a model into misclassifying images of people.

### Load raw dataset
- The dataset can be loaded using the *load_cifar100_file()* function from *utils.py*, see its perfectly written docstring.
- you should obtain at least *x_train, y_train, x_test, y_test*: the training/test data x and labels y.
- *classes* gives you the different classes for the dataset with the 20 super classes.
- **In order to have more efficient models, you should change the classes in order to have only people vs. things.**

In [304]:
#TODO


### Balance the dataset for training
- **restore the balance**: If you have changed to "people" or "things", the instances of each class are now unbalanced. Randomly select a subset of the "things" examples of the same size as the "people".
- Having balanced datasets for training is crucial, else you are building an algorithmic bias.

In [308]:
#TODO


<!-- - Optionnaly: you can select a random subset to decrease the size of the training and test datasets. You can use random_subsample from utils... But everything is fine. -->

In [313]:
print(f"Number of Training examples: {len(y_train)}")
print(f"Number of Test examples: {len(y_test)}")
print(f"Number of classes: {len(classes)}")

Number of Training examples: 840
Number of Test examples: 214
Number of classes: 2


### Visualize some images
One can use the show_images function from utils.py.

In [314]:
#TODO


### Visualize the dataset
- use dimensional reduction with PCA and plot the pictures in 2D
- plot the distribution of classes in the training and test datasets

In [316]:
#TODO


## 1.2 Our classification model: a classical CNN
We use a simple Convolutional Neural Network (CNN) architecture with:
- 3 convolutional blocks (each: convolution, max-pooling, dropout)
- 2 dense layers (one hidden layer + final output layer)

CNNs are the standard choice for image analysis, as they efficiently capture spatial patterns.
Here, a small architecture is sufficient since our images are small (32×32) and we only have 2 classes ("people" and "things").

### Create the model
- Study the function create_model_cifar from utils.py
- Use it to create your model

In [319]:
#TODO

### Fine-tune the model for the particular data
- This can be done with a simple call to the *fit* method of the model, look for the documentation of Tensorflow or examples: https://www.geeksforgeeks.org/deep-learning/model-fit-in-tensorflow/.
- Note that the images are already normalized: pixel values are between 0 and 1 so no preprocessing to do.

In [321]:
#TODO


## 1.3 Finally, let's start using ART ! Adversarial-Robustness Toolbox
A key idea behind ART is that it adopts an agnostic approach with respect to the underlying machine learning library.
This means ART is designed to work seamlessly with models implemented in different frameworks (e.g. Tensorflow, Pytorch, Scikit-learn)

*Why do we wrap models?*: In our case (using TensorFlow 2.x and Keras), we use:  **from art.estimators.classification import TensorFlowV2Classifier**. This wrapper converts our TensorFlow model into a generic ART classifier, which provides a unified API for: training, evaluation, adversarial example generation, defense setup, etc.

*Remark:* We could have chosen to wrap the model first, and then do the fine-tuning on the ART classifier object. In fact, we will now check the accuracy of the fine-tuned model using this interface.

### Check the accuracy of the model on the train and test set
- By applying the *predict* method of the classifier, and comparing with the truth for all the examples, you can get the overall accuracy for both datasets.

In [347]:
#TODO

- Compute the confusion matrix for the test set (you can use *scikit-learn.metrics*)

In [349]:
#TODO


- Test the model on a picture of your choice (resize it to 32x32, /255) and/or images from the test set

In [351]:
# TODO


# Part 2: Attack on machines
In this part, we focus on **evasion attacks**, where the attacker modifies inputs at test time to fool the classifier:
- **FGSM**, **PGD**: **white-box attacks**, where the attacker has full knowledge of the model.
- **Boundary Attack**: **black-box attack**, where the attacker can only query the model without knowing its details.

Here, we show but a few example attacks. Many more are available:
[ART attacks list](https://github.com/Trusted-AI/adversarial-robustness-toolbox/wiki/ART-Attacks)


## 2.1 FGSM (Fast Gradient Sign Method)
One of the simplest and most popular adversarial attack techniques. 
The idea is to create a small perturbation that pushes the input in the direction that maximally increases the model's loss.

The resulting perturbation is very small (almost imperceptible to the human eye), but can significantly change the model's prediction.


<img src="https://miro.medium.com/v2/resize:fit:1043/1*7BgNv9c2n7KNq3rrUcfZ8A.png" alt="Red and Blue teams for Adversarial ML" width = 500/>

See [Goodfellow et al., 2015](https://arxiv.org/abs/1412.6572).

### Generation of adversarial examples
In order to apply FGSM in ART (see notebooks/adversarial_training_mnist.ipynb for an example):
- create an object *FastGradientMethod*
- use its method *generate* for the genration of the adversarial examples based on the test set
- visualize the original image with its adversarial counterpart and the noise. One can use *plot_adversarial_comparison* from utils.py.

In [353]:
#TODO


### Impact on the model accuracy
- compute the accuracy on the model for the original test set and the adversarial examples.
- try to have the parameter epsilon vary from 0 to 0.2, plot the impact on these accuracies.

In [355]:
#TODO


**Questions:**
- How doeas the accuracy evolve with epsilon ?
- For the attacker, what is the risk of using high values for epsilon ?

## 2.2 Projected Gradient Descent (PGD)
The **Projected Gradient Descent (PGD)** attack is one of the most powerful and widely used to evaluate model robustness.

PGD can be seen as a stronger, multi-step version of FGSM. Instead of applying a single large gradient step, PGD takes multiple small steps, each time moving slightly in the direction that increases the loss. After each step, the perturbed image is **projected back** into a valid range around the original image, to keep the perturbation small.

By iterating and projecting, PGD finds more effective adversarial examples than single-step methods like FGSM.


<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ4RpYZnPeRzILqJAA1wnPXk8361OW6lpukcA&s" alt="Red and Blue teams for Adversarial ML" width = 500/>

See [Madry et al., 2018](https://arxiv.org/pdf/1706.06083).

### Generation and test
- Do the same as FSGM !

In [358]:
#TODO


###  Generate targetted adversarial examples
It is possible to specify what class you want to be predicted for the adversarial example: targetted adversarial examples.
In terms of the algorithm, we follow the inverse direction of the gradient, which will decrease the loss for the targeted label.
This is close to the classical gradient descent for training.

Generat target adversarial examples:
- This can be done with
    - the argument *targeted=True* in *ProjectedGradientDescent*
    - the argument *y* in *generate* which gives the wanted label(s)
- Compute the predictions for the entire adversarial dataset and display the confusion matrix

In [361]:
#TODO


**Questions:**
- Does the confusion matrix reflect the effect of the target adversarial generation ?

## 2.3 Black Box: Boundary Attack 
This is a black-box evasion attack: this attack **only needs to query the model and observe the output label**. We do not need to know the model's parameters or architecture.

The idea is to start from a **large initial adversarial example** (e.g., pure noise or an image that is already misclassified) and then **iteratively reduce the perturbation** while keeping the image adversarial.
The algorithm moves toward the original image while staying on the "wrong side" of the decision boundary. This way, it progressively finds a minimally perturbed adversarial example.

The attack can also be **targeted**, meaning it aims to change the prediction to a specific target class instead of just any incorrect class.

See a [nice explanation here](https://github.com/greentfrapp/boundary-attack) and the original paper [Brendel et al. 2018](https://arxiv.org/abs/1712.04248).

### Setting up the attack
It is a bit complicated to set up.
If you still have a lot of time, I advise looking at "notebooks/attack_decision_based_boundary.ipynb".
Else, you can get to the next part.


In [365]:
init_image_ind = 171
target_image_ind = 735
init_image=np.array([x_train[init_image_ind]])
target_image=np.array([x_train[target_image_ind]])

In [366]:
#TODO


# Part 3: Defending againt the machine
- We start with **detection**, which aims to identify whether an input is adversarial or clean before making a prediction.
- Then, we explore **adversarial training**, where the model is trained on adversarial examples to improve its robustness.

Again, these are just a few example approaches, but other defense methods are available in ART:
[ART defenses list](https://github.com/Trusted-AI/adversarial-robustness-toolbox/wiki/ART-Defences)

## 3.1 Detector based on the inputs
Detecting adversarial examples can be formulated as a **binary classification problem**, where the goal is to distinguish between **legitimate** and **adversarial** inputs.  

To build a detector, we can **generate adversarial examples from the training set** and label them as "adversarial," while keeping the original samples labeled as "legitimate."  

We then train a separate **CNN-based classifier** on this combined dataset to learn to identify adversarial perturbations directly from the input images.

### Set up the Detector
- Here, we will use a CNN which can be created with the function *create_model_cifar* from utils as in Part I.
- This model is wrapped in ART using *TensorFlowV2Classifier* as before
- Then, create a *BinaryInputDetector* based on this model, this is the object we use to detect adversarial examples.

In [370]:
#TODO


### Creating adversarial examples
Now you can create an attacker (e.g. *FastGradientMethod*) as before and generate adversarial examples on:
- the test dataset,
- the train dataset.

In [372]:
#TODO


### The actual detector
Now we can train and evaluate the detector model:
- concatenate the original train dataset (label legit/0) and the corresponding generated dataset (label adversarial/1)
- fit the detector based on the resulting training dataset
- compute the accuracy of detection on the test set (should be legit), and on the adverarial examples generated from the test set (should be adversarial)

In [374]:
#TODO


### Impact of epsilon
Without retraining your detection model:
- vary the value of epsilon between 0.01 and 0.9
- generate adversarial examples from the test set using this epsilon
- plot the number of adversarial examples flagged
- using the classifier for people/thing, plot the number of misclassified legit data

In [377]:
#TODO


## 3.2 Adversarial training
**Adversarial training** is a classical strategy to improve a model's robustness against attacks.  

The main idea is to **train the model not only on legitimate examples but also on adversarially perturbed examples**. By doing this, the model learns to correctly classify even when inputs are slightly manipulated.

In ART, this is implemented using the `AdversarialTrainer`, which automatically generates adversarial examples during training and includes them in each batch. This forces the model to adapt to adversarial perturbations and become more resistant to future attacks.


### Setup the classifiers
- Retrieve the classifier used for people/things and make a copy *robust_classifier* (for comparison purposes)
- Create a generator of advesarial examples based on PGD
- Create a *AdversarialTrainer* using the copy and the attack model as argument
- Launch a training with *fit* as usual => This is the actual adversarial training !

In [380]:
#TODO


### Compare the accuracy of the resulting model with the accuracy of the original model
Compute the accuracy of this classifier
- on the test set
- on adversarial examples generated using the previous attacker and a new one

In [386]:
#TODO


### Impact of epsilon
Without retraining your robust model:
- vary the value of epsilon between 0.01 and 0.9
- generate adversarial examples from the test set and the original and robust classifier
- plot the accuracy for both classifier

In [389]:
#TODO
